In [1]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Merge, Reshape, Dropout
from keras.layers.embeddings import Embedding
from keras.optimizers import SGD
from keras.layers.normalization import BatchNormalization

Using TensorFlow backend.


In [4]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from scripts.kaggle.helpers import make_submission
import datetime
from tqdm import tqdm
import gc
import itertools
from multiprocessing import Pool
import pickle

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

%pylab inline
pylab.rcParams['figure.figsize'] = (12, 16)

%load_ext autotime

Populating the interactive namespace from numpy and matplotlib


In [7]:
import sys
import numpy as np
import pandas as pd
from sklearn import preprocessing

time: 1.37 ms


In [73]:
def keras_model():
    # keras model architecture #
    final_model = Sequential()
    final_model.add(Dense(50, input_dim=train.shape[1], init='he_uniform'))
    final_model.add(Activation('relu'))
    final_model.add(Dense(train_y.shape[1], init='zero'))
    final_model.add(Activation('sigmoid'))
    final_model.compile(loss='binary_crossentropy', optimizer='adam')
    return final_model

time: 4.88 ms


In [5]:
description, train, added_products, label_encoder, encoders = pickle.load(open('/home/ubuntu/old_machine/home/ubuntu/santander/data/train_data.train_data', 'rb'))

time: 32.2 s


In [28]:
encoder_cols = ['sexo', 'indrel_1mes', 'pais_residencia', 'ind_empleado',
                'segmento', 'tiprel_1mes', 'indresi', 'indext', 'conyuemp',
                'canal_entrada', 'indfall', 'nomprov']

time: 1.36 ms


In [44]:
for col in encoder_cols:
    train[col] = encoder_dict[col].inverse_transform(train[col])
    

for col in encoder_cols:
    temp = pd.get_dummies(train[col])
    temp.columns = [col + '_' + x for x in temp.columns]
    train = pd.concat((train, temp), axis=1, join_axes=[train.index])

for col in encoder_cols:
    del train[col]

time: 4.32 s


In [53]:
train_y = pd.get_dummies(added_products.added_product)

time: 79.9 ms


In [80]:
train_index = train.index
train_y_index = train_y.index

time: 1.31 ms


In [69]:
train.fillna(0, inplace=True)

train.reset_index(inplace=True)
train_y.reset_index(inplace=True)
del train['ncodpers'], train_y['ncodpers']

train_np = np.array(train)

train_y_np = np.array(train_y)

time: 2.26 s


In [75]:
model = keras_model()

time: 139 ms


In [88]:
import time

time: 955 µs


In [132]:
# hist = model.fit(train_np, train_y_np, verbose=1)
# time.sleep(0.1)

# fit_generator(
# 			generator = batch_generator(train, 500, False), 
# 			nb_epoch = 1,
# 			samples_per_epoch = train_size
# 		)

hist= model.fit(train_np, train_y_np, nb_epoch=50, batch_size=train_np.shape[0])
time.sleep(0.1)

Epoch 1/50
441652/441652 [==============================] - 5s - loss: 0.6718
Epoch 2/50
441652/441652 [==============================] - 5s - loss: 0.6718
Epoch 3/50
441652/441652 [==============================] - 5s - loss: 0.6718
Epoch 4/50
441652/441652 [==============================] - 5s - loss: 0.6718
Epoch 5/50
441652/441652 [==============================] - 5s - loss: 0.6718
Epoch 6/50
441652/441652 [==============================] - 5s - loss: 0.6718
Epoch 7/50
441652/441652 [==============================] - 5s - loss: 0.6718
Epoch 8/50
441652/441652 [==============================] - 5s - loss: 0.6718
Epoch 9/50
441652/441652 [==============================] - 5s - loss: 0.6718
Epoch 10/50
441652/441652 [==============================] - 5s - loss: 0.6718
Epoch 11/50
441652/441652 [==============================] - 7s - loss: 0.6718
Epoch 12/50
441652/441652 [==============================] - 7s - loss: 0.6718
Epoch 13/50
441652/441652 [==============================] - 

KeyboardInterrupt: 

time: 2min 47s


In [130]:
hist= model.fit_generator?

time: 3.36 ms


In [ ]:
hist= model.fit_generator

In [ ]:
hist= model.fit

In [91]:
description, test1 = pickle.load(open('/home/ubuntu/old_machine/home/ubuntu/santander/data/13.test_data1.test_data1', 'rb'))
# test1.drop([500000], inplace=True)
# description, test2 = pickle.load(open('/home/ubuntu/old_machine/home/ubuntu/santander/data/13.test_data2.test_data2', 'rb'))
# test = pd.concat((test1, test2))
# test.set_index('ncodpers', inplace=True)
# del test1, test2
# gc.collect()
# test.reset_index(inplace=True)

time: 33 s


In [92]:
for col in encoder_cols:
    test1[col] = encoder_dict[col].inverse_transform(test1[col])
    

# for col in encoder_cols:
    temp = pd.get_dummies(test1[col])
    temp.columns = [col + '_' + x for x in temp.columns]
    test1 = pd.concat((test1, temp), axis=1, join_axes=[test1.index])

# for col in encoder_cols:
    del test1[col]

time: 24.1 s


In [93]:
test1_index = test1.index

time: 849 µs


In [94]:
test1.fillna(0, inplace=True)

test1.reset_index(inplace=True)
del test1['ncodpers']

non_existent_test_cols = list(set(train.columns).difference(set(test1.columns)))

for col in non_existent_test_cols:
    test1[col] = [0.0]*test1.shape[0]

test1 = test1[train.columns]

test1_np = np.array(test1)


time: 4.97 s


In [116]:
preds = model.predict_proba(test1_np)

499904/500001 [============================>.] - ETA: 0stime: 10.7 s


In [117]:
preds.shape

(500001, 24)

time: 2.9 ms


In [120]:
preds[3566]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.], dtype=float32)

time: 3.61 ms


In [121]:
preds_tr = model.predict_proba(train_np)

441600/441652 [============================>.] - ETA: 0stime: 9.24 s


In [123]:
preds_classes = model.predict_classes(test1_np)

499776/500001 [============================>.] - ETA: 0stime: 10.7 s


In [124]:
preds_classes.shape

(500001,)

time: 2.69 ms


In [ ]:
l = list(preds_classes)

In [133]:
from sklearn.datasets import load_iris

time: 1 ms


In [135]:
iris = load_iris()

time: 150 ms


In [137]:
iris.data

array([[ 5.1,  3.5,  1.4,  0.2],
       [ 4.9,  3. ,  1.4,  0.2],
       [ 4.7,  3.2,  1.3,  0.2],
       [ 4.6,  3.1,  1.5,  0.2],
       [ 5. ,  3.6,  1.4,  0.2],
       [ 5.4,  3.9,  1.7,  0.4],
       [ 4.6,  3.4,  1.4,  0.3],
       [ 5. ,  3.4,  1.5,  0.2],
       [ 4.4,  2.9,  1.4,  0.2],
       [ 4.9,  3.1,  1.5,  0.1],
       [ 5.4,  3.7,  1.5,  0.2],
       [ 4.8,  3.4,  1.6,  0.2],
       [ 4.8,  3. ,  1.4,  0.1],
       [ 4.3,  3. ,  1.1,  0.1],
       [ 5.8,  4. ,  1.2,  0.2],
       [ 5.7,  4.4,  1.5,  0.4],
       [ 5.4,  3.9,  1.3,  0.4],
       [ 5.1,  3.5,  1.4,  0.3],
       [ 5.7,  3.8,  1.7,  0.3],
       [ 5.1,  3.8,  1.5,  0.3],
       [ 5.4,  3.4,  1.7,  0.2],
       [ 5.1,  3.7,  1.5,  0.4],
       [ 4.6,  3.6,  1. ,  0.2],
       [ 5.1,  3.3,  1.7,  0.5],
       [ 4.8,  3.4,  1.9,  0.2],
       [ 5. ,  3. ,  1.6,  0.2],
       [ 5. ,  3.4,  1.6,  0.4],
       [ 5.2,  3.5,  1.5,  0.2],
       [ 5.2,  3.4,  1.4,  0.2],
       [ 4.7,  3.2,  1.6,  0.2],
       [ 4

time: 20.4 ms


In [139]:
iris.target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

time: 3.88 ms


In [140]:
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(4, input_dim=4, init='normal', activation='relu'))
	model.add(Dense(3, init='normal', activation='sigmoid'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

time: 5.67 ms


In [141]:
iris_model = baseline_model()

time: 143 ms


In [152]:
iris_ohe = preprocessing.OneHotEncoder()
iris_ohe.fit(iris.target.reshape(-1,1))
iris_traget = iris_ohe.transform(iris.target.reshape(-1,1))

time: 4.59 ms


In [154]:
print iris_traget.shape
print iris.data.shape

(150, 3)
(150, 4)
time: 1.63 ms


In [157]:
iris_traget = iris_traget.todense()

time: 2.22 ms


In [158]:
hist = iris_model.fit(iris.data, iris_traget)

Epoch 1/10
150/150 [==============================] - 0s - loss: 1.0979 - acc: 0.3333     
Epoch 2/10
150/150 [==============================] - 0s - loss: 1.0975 - acc: 0.3333     
Epoch 3/10
150/150 [==============================] - 0s - loss: 1.0972 - acc: 0.3467     
Epoch 4/10
150/150 [==============================] - 0s - loss: 1.0967 - acc: 0.3667     
Epoch 5/10
150/150 [==============================] - 0s - loss: 1.0963 - acc: 0.5067     
Epoch 6/10
150/150 [==============================] - 0s - loss: 1.0959 - acc: 0.5333     
Epoch 7/10
150/150 [==============================] - 0s - loss: 1.0953 - acc: 0.5333     
Epoch 8/10
150/150 [==============================] - 0s - loss: 1.0948 - acc: 0.4733     
Epoch 9/10
150/150 [==============================] - 0s - loss: 1.0941 - acc: 0.4267     
Epoch 10/10
150/150 [==============================] - 0s - loss: 1.0934 - acc: 0.3867     
time: 300 ms


In [159]:
iris_model.predict_classes(iris.data)

150/150 [==============================] - 0s     


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

time: 52.7 ms


In [160]:
iris_model.predict_proba(iris.data)

 32/150 [=====>........................] - ETA: 0s

array([[ 0.51247084,  0.4967038 ,  0.49845058],
       [ 0.51057971,  0.49730912,  0.4985933 ],
       [ 0.51137215,  0.4970555 ,  0.49853352],
       [ 0.51039195,  0.49736926,  0.49860746],
       [ 0.51272935,  0.4966211 ,  0.49843103],
       [ 0.51303113,  0.49652445,  0.49840823],
       [ 0.51157624,  0.4969902 ,  0.49851811],
       [ 0.51176602,  0.49692944,  0.49850377],
       [ 0.50980812,  0.49755606,  0.49865156],
       [ 0.51077163,  0.49724767,  0.49857879],
       [ 0.51313996,  0.49648967,  0.49839997],
       [ 0.51131976,  0.49707231,  0.49853742],
       [ 0.51061738,  0.49729708,  0.49859047],
       [ 0.51101583,  0.49716952,  0.4985604 ],
       [ 0.51533931,  0.49578553,  0.49823391],
       [ 0.51555759,  0.49571559,  0.4982174 ],
       [ 0.51413208,  0.49617204,  0.49832514],
       [ 0.5123477 ,  0.4967432 ,  0.49845982],
       [ 0.51306677,  0.49651307,  0.49840555],
       [ 0.51310438,  0.49650106,  0.49840271],
       [ 0.51155746,  0.49699619,  0.498

time: 27.8 ms
